In [1]:
import os,shutil    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import numpy as np
import torch
import os.path
import glob
import pandas as pd
import random
from math import sqrt,pi

import pandas as pd

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from torchvision.utils import make_grid, save_image
from torch.utils.tensorboard import SummaryWriter

In [25]:
from tqdm import tqdm

# Att processing

## Att functions

In [2]:
%cd "idgan-master"

C:\Users\zhang\idgan-master


In [3]:
import argparse
import os
from os import path
import copy
from tqdm import tqdm
import torch
from torch import nn
from gan_training import utils
from gan_training.checkpoints import CheckpointIO
from gan_training.distributions import get_ydist, get_zdist
from gan_training.eval import Evaluator
from gan_training.config import (
    load_config, build_models
)

In [4]:
def get_one_hot(label, N):
    size = list(label.size())
    label = label.view(-1).cpu()   #reshape to a long vector
    ones = torch.sparse.torch.eye(N)
    ones = ones.index_select(0, label)   #turn to one hot
    size.append(N)  #reshape to h*w*channel classes
    return ones.view(*size).squeeze(1).permute(0,3,1,2)

def discretize_to_order_labels(t):
    tensor=t.cpu()
    bins = torch.tensor([-0.125,0.125, 0.375, 0.625, 0.875,1.125])
    inds = torch.bucketize(tensor, bins)
    tensor_discret = inds.add(-1)
    
    return tensor_discret

def generator_postprocess(x):
    n_channels = x.size(1)
    
    if n_channels ==5:
        x_im = x.add(1).div(2).clamp(0,1).round().argmax(dim=1).unsqueeze(1).cpu()/(n_channels-1)
        return x_im
        
    else:
        x_shift = x.add(1).div(2).cpu()
    
    if n_channels ==5:
        x_recover = x.softmax(dim=1).argmax(dim=1).unsqueeze(1).cpu()/(n_channels-1)
        return x_recover
        
    else:
        return torch.sigmoid(x)
    
def load_simple_im(path):
    example =  cv2.imread(path)
    
    gt = np.mean(example,axis=2)/256
    
    gt = (gt-0.5)*2

    img_tensor = torch.from_numpy(gt).unsqueeze(0).float()
    
    return img_tensor

def get_nsamples(data_loader, N):
    x = []
    y = []
    n = 0
    while n < N:
        x_next, y_next = iter(data_loader).next()
        #x_next = iter(data_loader).next()
        x.append(x_next)
        y.append(y_next)
        n += x_next.size(0)
    x = torch.cat(x, dim=0)[:N]
    y = torch.cat(y, dim=0)[:N]
    return x, y
    #return x

def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')    
    
def generator_postprocess(x):
    n_channels = x.size(1)
    
    if n_channels ==5:
        x_im = x.add(1).div(2).clamp(0,1).round().argmax(dim=1).unsqueeze(1).cpu()/(n_channels-1)
        return x_im
        
    else:
        x_shift = x.add(1).div(2).cpu()
        x_discret = discretize_to_order_labels(x_shift)

        return x_discret.div(n_channels-1)

def decoder_postprocess(x):
    n_channels = x.size(1)
    
    if n_channels ==5:
        x_recover = x.softmax(dim=1).argmax(dim=1).unsqueeze(1).cpu()/(n_channels-1)
        return x_recover
        
    else:
        return torch.sigmoid(x)

In [5]:
class solargan_im_trainset(Dataset):
    def __init__(self,images, loader):
        
        self.images = images #image path
        self.loader = loader

    def __getitem__(self, index):
        fn = self.images[index]
        tensor = self.loader(fn)
        return tensor, fn

    def __len__(self):
        return len(self.images)

    def save_samples(self, N, root='./true_samples'):
        from torchvision.utils import save_image
        save_dir = os.path.join(root, self.type)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        sample_idxs = random.choices(list(range(len(self))), k=N)
        for i, sample_idx in enumerate(sample_idxs):
            path = os.path.join(save_dir, '{}.jpg'.format(i+1))
            save_image(self.__getitem2__(sample_idx), path, 1, 0)

In [6]:
def get_dataset(name, data_dir, size=128, lsun_categories=None):
    transform = transforms.Compose([
        transforms.Resize(size),
        transforms.CenterCrop(size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        transforms.Lambda(lambda x: x + 1./128 * torch.rand(x.size())),
    ])

    if name == 'solargan_im':
        grayimg_list = glob.glob(data_dir+"/*.png")
        dataset = solargan_im_trainset(grayimg_list,load_simple_im)
    elif name == 'solargan_im_res':
        grayimg_list = glob.glob(data_dir+"/*.png")
        dataset = solargan_im_trainset(grayimg_list,load_simple_im)
    elif name == 'solargan':
        grayimg_list = glob.glob(data_dir+"/*.png")
        dataset = solargan_im_trainset(grayimg_list,gray_label_to_torch_tensor)
    else:
        raise NotImplementedError

    return dataset

In [7]:
#configres_path = 'seasia_im_single.yaml'
configres_path = 'sbe_imtest_single.yaml'

#configs
configres = load_config(configres_path)

c_dim = configres['dvae']['c_dim']
out_res_name = configres['test']['out_name']
checkpoint_res_dir = path.join(out_res_name, 'chkpts')
batch_size = configres['test']['batch_size']

c_dim_wwr = 30

In [8]:
#load_batch_samples
test_dataset = get_dataset(
    name=configres['data']['type'],
    data_dir=configres['data']['test_dir'],
    size=configres['data']['img_size'],
)
test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False, pin_memory=True, sampler=None, drop_last=True
)


In [9]:
# Logger
checkpoint_res_io = CheckpointIO(
    checkpoint_dir=checkpoint_res_dir
)


# Create models
dvae, generator_res, discriminator_res = build_models(configres)
dvae_ckpt_path = os.path.join('outputs', configres['dvae']['runname'], 'chkpts', configres['dvae']['ckptname'])
dvae_ckpt = torch.load(dvae_ckpt_path)['model_states']['net']
dvae.load_state_dict(dvae_ckpt)

# Put models on gpu if needed

is_cuda = torch.cuda.is_available()

device = torch.device("cuda:0" if is_cuda else "cpu")

dvae = dvae.to(device)
generator_res = generator_res.to(device)
discriminator_res = discriminator_res.to(device)


# Use multiple GPUs if possible
generator_res = nn.DataParallel(generator_res)
discriminator_res = nn.DataParallel(discriminator_res)

# Register modules to checkpoint
checkpoint_res_io.register_modules(
    generator=generator_res,
    discriminator=discriminator_res,
)


# Test generator
generator_res_test = generator_res

# Distributions
zdist = get_zdist(configres['z_dist']['type'], configres['z_dist']['dim'],
                  device=device)
z = zdist.sample((batch_size,))


# Load checkpoint if existant
it_res = checkpoint_res_io.load('model_00299999.pt')

=> Loading checkpoint...


In [10]:
# read att file
#att = pd.read_csv('att_seasia_splitted_train.csv')
att = pd.read_csv('sbe_att.csv')

In [13]:
for x_real_test_batch,filename in test_loader:
    
    im_index = str(filename)[34:][:-6] ##edit if bug
    
    print(im_index)
    
    x_real_shift = x_real_test_batch.add(1).div(2)


    if x_real_shift.size(1) == 1:
        x_real_disc = discretize_to_order_labels(x_real_shift)
        x_real_onehot = get_one_hot(x_real_disc, 5)
        x_real_onehot = x_real_onehot.to(device)

        c, c_mu, c_logvar = cs = dvae(x_real_onehot, encode_only=True)

    else:
        x_real_shift = x_real_shift.to(device)
        c, c_mu, c_logvar = cs = dvae(x_real_shift, encode_only=True)
    
    ix =att[(att['idx']==int(im_index))].index
    att.loc[ix,'im_0':'im_31']=c_mu.cpu().detach().numpy().squeeze()
    att.loc[ix ,'var_0':'var_31']=np.exp(c_logvar.cpu().detach().numpy().squeeze())

0
1
10
100
101
102
103
104
105
106
107
108
109
11
110
111
112
113
114
115
116
117
118
119
12
120
121
122
123
124
125
126
127
128
129
13
130
131
132
133
134
135
136
137
138
139
14
140
141
142
143
144
145
146
147
148
149
15
150
151
152
153
154
155
156
157
158
159
16
160
161
162
163
164
165
166
167
168
169
17
170
171
172
173
174
175
176
177
178
179
18
180
181
182
183
184
185
186
187
188
189
19
190
191
192
193
194
195
196
197
198
199
2
20
200
201
202
203
204
205
206
207
208
209
21
210
211
212
213
214
215
216
217
218
219
22
220
221
222
223
224
225
226
227
228
229
23
230
231
232
233
234
235
236
237
238
239
24
240
241
242
243
244
245
246
247
248
249
25
250
251
252
253
254
255
256
257
258
259
26
260
261
262
263
264
265
266
267
268
269
27
270
271
272
273
274
275
276
277
278
279
28
280
281
282
283
284
285
286
287
288
289
29
290
291
292
293
294
295
296
297
298
299
3
30
300
301
302
303
304
305
306
307
308
309
31
310
311
312
313
314
315
316
317
318
319
32
320
321
322
323
324
325
326
327
328
329
33


In [14]:
#att.to_csv('att_seasia_train_im_lat.csv',index=False)
att.to_csv('sbe_im_lat.csv',index=False)

In [15]:
%cd ..

C:\Users\zhang


## Keep relevant atts

In [16]:
%cd "att_feat_processing"

C:\Users\zhang\att_feat_processing


In [17]:
sbe_att_all = pd.read_csv('sbe_im_lat.csv')
sbe_att_all.drop(labels=['idx','ptix','norm_z','glzr'],axis=1,inplace=True)
sbe_att_rel = sbe_att_all.loc[:,:'im_31']
sbe_att_rel.to_csv('att_rel_sbe_im_lat.csv',index=False)

In [26]:
def finalize_train_data(loc_type,att_path, start_ix, end_ix):
    
    if loc_type == 'europe':
        site_list = ['geneva','milan','paris','perlin','zurich']
        lat_long_list = np.array([[46.25,6.13],[45.43,9.28],[48.73,2.40],[52.47,13.40],[47.480,8.536]])
        att_all = pd.read_csv(att_path)
        att = att_all.loc[start_ix:end_ix,:]
        feature_path = 'europe_features/'
        weather_path = 'europe_weather/'

    elif loc_type == 'seasia':
        site_list = ['hochiminh','jakarta','kualalumpur','pangkok','singapore']
        lat_long_list = np.array([[10.81900,106.6520],[-6.18300,106.8330],[3.12,101.55],[13.92,100.60],[1.37,103.98]])
        att_all = pd.read_csv(att_path)
        att = att_all.loc[start_ix:end_ix,:]
        feature_path = 'seasia_features/'
        weather_path = 'seasia_weather/'
    
    mon = pd.read_csv('week_mon.csv',header=None)
    mon=mon.values.reshape(52,1)

    week_day = np.arange(52)*7+3.5
    week_day = week_day.reshape(52,1)
    inc_lowb = np.sin((week_day-7+284)/365.25*2*np.pi)
    inc_highb = np.sin((week_day+7+284)/365.25*2*np.pi)

    inc = inc_lowb+(np.random.rand(52,1)*(inc_highb-inc_lowb))
    
    sample_num = end_ix-start_ix+1
    lat_long_all =np.tile(lat_long_list[0,:],(sample_num*52,1))

    lat_long_all=np.append(lat_long_all,np.tile(lat_long_list[1,:],(sample_num*52,1))).reshape(-1,2)
    lat_long_all=np.append(lat_long_all,np.tile(lat_long_list[2,:],(sample_num*52,1))).reshape(-1,2)
    lat_long_all=np.append(lat_long_all,np.tile(lat_long_list[3,:],(sample_num*52,1))).reshape(-1,2)
    lat_long_all=np.append(lat_long_all,np.tile(lat_long_list[4,:],(sample_num*52,1))).reshape(-1,2)
    
    train_att_list = np.empty(shape=(0,45)) #ori: 35 + 8 weather + 2 long/lat +1 mon_ix + 1 solar inc
    train_feat_list = np.empty(shape=(0,119)) #17 * 7 hours with sunlight

    for site in site_list:    
        weather = pd.read_csv(weather_path+site+'_stat.csv',header=None)

        weather_stat = weather.values
        train_att_site_list = np.empty(shape=(0,45))
        train_feat_site_list = np.empty(shape=(0,119))


        for ptix in tqdm(range(start_ix, end_ix+1)):

            tain_att_site_sample = np.empty(shape=(52,35))
            feature_site_sample = np.empty(shape=(52,119))

            tain_att_site_sample[0:52,:]=att.loc[ptix,:].values
            
            ix_feature_path = feature_path+site+'/'+str(ptix)+'.csv'
            feature_ix = np.loadtxt(ix_feature_path,delimiter = ",")
            feature_ix_day = (feature_ix[:8736].reshape(-1,24))[:,4:21] #sunlit hours
            feature_ix_day_weekly = feature_ix_day.reshape(-1).reshape(52,-1)

            feature_site_sample[0:52,:] = feature_ix_day_weekly

            inc = inc_lowb+(np.random.rand(52,1)*(inc_highb-inc_lowb))
            date_inc = np.concatenate((mon,inc),axis=1)

            tain_att_site_sample=np.concatenate((tain_att_site_sample,date_inc),axis=1)
            tain_att_site_sample=np.concatenate((tain_att_site_sample,weather_stat),axis=1)

            train_att_site_list = np.append(train_att_site_list,tain_att_site_sample).reshape(-1,45)
            train_feat_site_list = np.append(train_feat_site_list,feature_site_sample).reshape(-1,119)

        train_att_list = np.append(train_att_list,train_att_site_list).reshape(-1,45)
        train_feat_list = np.append(train_feat_list,train_feat_site_list).reshape(-1,119)
        
    train_att_list_all = np.concatenate((lat_long_all,train_att_list),axis=1)

    return train_att_list_all, train_feat_list



In [27]:
att_sbe, feat_sbe =  finalize_train_data('europe','att_rel_sbe_im_lat.csv', 0, 639)

100%|██████████| 640/640 [00:12<00:00, 51.37it/s]


In [28]:
np.save('sbe_att.npy',att_sbe)
np.save('sbe_feat.npy',feat_sbe)